In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from animalclef.spark import get_spark
from pyspark.sql import functions as F, Window
from pathlib import Path
from animalclef.dataset import split_reid_data, summarize_split


spark = get_spark(cores=4, memory="2g")

root = Path.home() / "scratch/animalclef"
metadata = spark.read.csv(f"{root}/raw/metadata.csv", header=True, inferSchema=True)
embeddings = spark.read.parquet(f"{root}/processed/embeddings.parquet")
embeddings = embeddings.select("image_id", "embeddings")
df = metadata.join(embeddings, on="image_id", how="inner").select(
    "image_id",
    "identity",
    "dataset",
    "embeddings",
    F.count("image_id").over(Window.partitionBy("identity")).alias("identity_count"),
)
df.printSchema()
pdf = df.toPandas()

25/05/12 18:15:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- image_id: integer (nullable = true)
 |-- identity: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- embeddings: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- identity_count: long (nullable = false)



In [8]:
cond = (~pdf.identity.isnull()) & (pdf.identity_count > 2)
train_df, val_df, test_df = split_reid_data(
    pdf[cond], train_ratio=0.6, val_ratio=0.5, known_ratio=0.8
)
display(summarize_split(train_df, val_df, test_df))

,Split,Num Individuals,Num Images,Train Image Overlap,Val Image Overlap,Test Image Overlap,Train Image %,Val Image %,Test Image %,Known Individuals,Unknown Individuals
0,Train,404,5442,5442,0,0,100.0,0.0,0.0,404,0
1,Validation,539,3364,0,3364,0,0.0,100.0,0.0,404,135
2,Test,425,3729,0,0,3729,0.0,0.0,100.0,290,135


In [9]:
cond = (~pdf.identity.isnull()) & (pdf.identity_count > 2)
train_df, val_df, test_df = split_reid_data(
    pdf[cond], train_ratio=0.8, val_ratio=0.5, known_ratio=0.8
)
display(summarize_split(train_df, val_df, test_df))

,Split,Num Individuals,Num Images,Train Image Overlap,Val Image Overlap,Test Image Overlap,Train Image %,Val Image %,Test Image %,Known Individuals,Unknown Individuals
0,Train,539,7671,7671,0,0,100.0,0.0,0.0,539,0
1,Validation,606,2231,0,2231,0,0.0,100.0,0.0,539,67
2,Test,447,2633,0,0,2633,0.0,0.0,100.0,379,68
